## Plane Shear Dispersion with Localized Initial Condition

Initial condition given by a gaussian centered at $x=0$. Rectangular domain is

\begin{equation}
-2\pi\leq x \leq 2\pi
\end{equation}

and so the smallest mode in the initial condition (besides $k=0$) that can fit in the domain is $k=0.5$. In paper, domain is larger, but it takes longer / uses more memory to run. This is for illustrative purposes.


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import holoviews as hv
hv.extension('bokeh')

In [ ]:
from mathieu_functions import A_coefficients
from mathieu_functions import mathieu_functions as mfs

In [ ]:
# =================================
# Important parameters to define
# =================================
L = np.pi # Half of channel width (y-direction)
N = 50  # length of k-array
alpha = 2  # length of channel periodic in x. I have used alpha=10 before, but for the gaussian initial condition a value of 2 is better.
Nx = 500  # length of x-array
sigma=0.5  # changes width of gaussian

eps = 0.05  # ta / td << 1 for weakly diffusive processes.
Pe = 1 / eps

x = np.linspace(-alpha * L, alpha * L, Nx)
y = np.linspace(0, L, Nx//5)
X, Y = np.meshgrid(x, y)

K = np.arange(0, N / alpha, 1 / alpha)  # wavenumber array.
K_test = np.linspace(0, N/alpha, 1000)
Q = (1j) * 2 * K / eps  # Canonical Mathieu parameter
qf = Q[-1].imag  # Largest value of Mathieu's parameter. 

M = 75  # matrix size
t = np.linspace(0, .25, 100)

In [ ]:
K

In [ ]:
if qf > 1000:
    print('Value of parameter q is:', (qf * (1j)))
    raise Warning('Change either epsilon or k, to reduce the size of q. The current code only works for values q>1000i')
print('Value of parameter q is:', (qf * (1j)))

In [ ]:
A_vals = A_coefficients(Q, M, 'even', 'one')
vals = mfs.ce_even(Q, y, M, As=A_vals)

## Mathieu Eigenfunctions

Creates a list with the right size

In [ ]:
CE = []  # Initialize list containing Mathieu Eigenfunctions
for k in range(M // 2):
    ce = np.repeat(vals['ce'+str(2 * k)][:, :, np.newaxis], Nx, axis=2)
    CE.append(ce)

COS = [np.exp(K[i] * X *(1j)) for i in range(N)]

In [ ]:
del ce

## Fourier Coefficients for the x-Fourier approximation of the initial condition

In [ ]:
fac = np.sqrt(np.pi)*sigma/(2*L)
arg = ((2 * L) / (np.pi*sigma))**2
cn = []
for n in range(N):
    cn.append(fac * np.exp(-K[n]**2/(arg**2)))

## Defines a function that constructs the solution 

In [ ]:
def evolve_ds(As, CE, K, cn, sigma, X, Y, t):
    """Constructs the solution to the IVP"""
    ## Initialize the array
    coords = {"time": t, 
              "y": Y[:, 0], 
              "x": X[0, :]}
    Temp = xr.DataArray(np.nan, coords=coords, dims=["time", 'y', 'x'])
    ds = xr.Dataset({'Theta': Temp})
    N = len(K)
    for i in range(len(t)):
        print(i)
        coeff=[]
        for k in range(N):
            CE2n = [2 * As['A'+str(2*r)][k, 0] * CE[r][k, :, :] * np.exp(-(0.25*As['a'+str(2*r)][k] + K[k]**2)*t[i]) for r in range(len(CE))]
            CE2n = sum(CE2n) # r-sum
            coeff.append(cn[k] * CE2n * COS[k])
        T0 = (sigma**2)*np.sum(coeff, axis=0).real # k-sum
        ds['Theta'].data[i, :, :] = T0
    return ds

## Construct solution


In [ ]:
ds = evolve_ds(A_vals, CE, K, cn, sigma, X, Y, t)

In [ ]:
del CE  # frees up memory

## Plot animation

In [ ]:
%%output holomap='scrubber'
%%opts Image style(cmap='nipy_spectral') plot[colorbar=True]
%%opts Image [width=600, height=450]
hv_ds = hv.Dataset(ds.Theta)
hv_ds.to(hv.Image, ['x', 'y'])

## Now calculate Mean

In [ ]:
Time, Xt = np.meshgrid(x, t)

In [ ]:
Tm = np.trapz(ds.Theta, axis=1) * y[1] / np.pi

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8), facecolor='w')
cf=plt.contourf(Time, Xt, Tm,  levels=np.linspace(0, 1, 1000), cmap='nipy_spectral')
plt.xticks(size=15)
plt.yticks([0, .1, .2], size=15)
plt.ylim(0, 0.2)
plt.xlabel('x', fontsize=25)
plt.xlim(-1.5*L, 1.5*L)
plt.ylabel(r'$\frac{t}{t_{d}}$', fontsize=35, rotation=0, labelpad = 35)
cbaxes = fig.add_axes([0.675, 0.935, 0.225, 0.03])
clb1 = plt.colorbar(cf,cax=cbaxes,ticks=[0, 0.5, 1],orientation='horizontal')
clb1.ax.tick_params(labelsize=15)
plt.show()